# Graph

In [1]:
import tensorflow as tf
import numpy as np

상수를 더하기 예제

In [ ]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0)

total = a + b

# tensorboard를 위해 그래프 추가
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

with tf.Session() as sess:
    print(sess.run({'ab':(a, b), 'total':total}))

그래프의 동작 방식을 이해하기 위한 예제이다. 다음은 그래프가 3번 실행 된다.
마지막 실행에서 out1, out2가 동일한 vec값을 사용했다는 점을 확인하기 바란다.

In [ ]:
#변수 3개 반환
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2

with tf.Session() as sess:
    print('vec: {}'.format(sess.run(vec)))
    print('vec: {}'.format(sess.run(vec)))
    # out1, out2가 동일한 vec을 사용했다
    print('out: {}'.format(sess.run((out1, out2))))

입력 값을 넣기 위해 placeholder를 추가한 내용이다.

In [ ]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

z = x + y
with tf.Session() as sess:
    print(sess.run(z, feed_dict={x: 3, y: 4.5}))
    print(sess.run(z, feed_dict={x: [1, 3], y:[2, 4]}))

## Dataset

In [23]:
source = np.array([
    [0, 1],
    [2, 3],
    [4, 5],
    [6, 7],
])

slices = tf.data.Dataset.from_tensor_slices(my_data)
iterator = slices.make_one_shot_iterator()
next_item = iterator.get_next()

sess = tf.Session()

while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break
        

[0 1]
[2 3]
[4 5]
[6 7]


In [24]:
my_data = tf.random_normal([10, 3])
slices = tf.data.Dataset.from_tensor_slices(my_data)
iterator = slices.make_initializable_iterator()
next_row = iterator.get_next()

with tf.Session() as sess:
    sess.run(iterator.initializer)
    
    i = 0
    while True:
        try:
            i+=1
            print(i, sess.run(next_row))
        except tf.errors.OutOfRangeError:
            break

1 [ 0.4575673  2.846385  -2.4032662]
2 [-0.38465163  0.5436408  -1.5063286 ]
3 [ 1.4354113  -0.63497657 -1.0856948 ]
4 [0.10020169 0.51569295 0.27205357]
5 [ 1.1109571  0.686919  -2.1738098]
6 [-1.0901219  -0.86027545  0.5627439 ]
7 [-0.49117967  0.08140505  0.7198881 ]
8 [-0.5775051  -0.12095883 -0.45476007]
9 [-1.1111729  -0.98800325  0.3664406 ]
10 [0.53594863 0.17011186 0.258575  ]


In [26]:
# 입력 소스로 부터 dataset을 구성
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4, 10], minval=0, maxval=100))
print(dataset1.output_types)
print(dataset1.output_shapes)

# 다른 입력 소스를 묶어서 dataset을 구성
dataset2 = tf.data.Dataset.from_tensor_slices(
    (tf.random_uniform([4]), tf.random_uniform([4, 100]))
)

print(dataset2.output_types)
print(dataset2.output_shapes)

# dataset을 묶어서 새로운 dataset을 구성
dataset3 = tf.data.Dataset.zip(
    (dataset1, dataset2)
)
print(dataset3.output_types)
print(dataset3.output_shapes)


<dtype: 'float32'>
(10,)
(tf.float32, tf.float32)
(TensorShape([]), TensorShape([Dimension(100)]))
(tf.float32, (tf.float32, tf.float32))
(TensorShape([Dimension(10)]), (TensorShape([]), TensorShape([Dimension(100)])))


In [19]:
# 딕셔너리를 이용한 입력소스 지정 방법. key에 이름을 부여할 수 있음
sources = dict()

sources['a'] = tf.random_uniform([4])
sources['b'] = tf.random_uniform([4, 100])

dataset = tf.data.Dataset.from_tensor_slices(sources)

print(dataset.output_types)
print(dataset.output_shapes)


{'a': tf.float32, 'b': tf.float32}
{'a': TensorShape([]), 'b': TensorShape([Dimension(100)])}


### Iterator

In [36]:
# one-shot iterator
# initialization이 필요 없다.

dataset = tf.data.Dataset.range(10)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    for i in range(10):
      print(sess.run(next_element))

0
1
2
3
4
5
6
7
8
9


In [45]:
# initalizable iterator
# 사용전, initialization 필수
# tf.placeholder를 이용해서 동작을 지정할 있는 장점이 있다.

max_value = tf.placeholder(tf.int64, shape=[])
dataset = tf.data.Dataset.range(max_value)

iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={max_value: 20})
    for i in range(10):
        print(sess.run(next_element))
    
with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={max_value:5})
    for i in range(5):
        print(sess.run(next_element))


0
1
2
3
4
5
6
7
8
9
0
1
2
3
4


In [47]:
# reinitilizable iterator
# 여러 Data source로 부터 데이터를 가지고 올수 있다.

training_dataset = tf.data.Dataset.range(100)
training_dataset.map(lambda x: x + tf.random_uniform([], -10, 10, tf.int64))
validation_dataset = tf.data.Dataset.range(100)

# reinitializable iterator의 경우는 structure로 정의된다.
# 다른 Data source라 할지라도 같은 데이터 형태를 가지기 때문에 둘 중 하나를 사용하면 된다.
iterator = tf.data.Iterator.from_structure(training_dataset.output_types, training_dataset.output_shapes)
next_element = iterator.get_next()

iterator = 


